# Tutorial: Building a Browser Use Agent From Scratch and with Magentic-UI



You might have seen cool video demos online of AI agents taking control of a computer or a browser to perform tasks. This is a new category of agents referred to as Computer-Use-Agents (CUA) or Browser-Use-Agents (BUA). Examples of such CUA/BUA agents include [OpenAI's Operator](https://openai.com/index/introducing-operator/), [Claude Computer Use Model](https://docs.anthropic.com/en/docs/agents-and-tools/tool-use/computer-use-tool), [AutoGen's MultiModalWebSurfer](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.agents.web_surfer.html), [Adept AI](https://www.adept.ai/blog/act-1), [Google's Project Mariner](https://deepmind.google/models/project-mariner/) and [Browser-Use](https://github.com/browser-use/browser-use/tree/main) among many others.


## What is a Computer Use Agent?

**Definition**: A computer or browser use agent is an agent that given a task, e.g., "order a shawarma sandwich from BestShawarma for pickup now", can programmatically control a computer or browser to autonomously complete the task. By "control a browser" we mean interacting with the browser in a similar way to how a human might control the browser: clicking on buttons, typing in fields, scrolling and so on. Note that a tool-use language model agent could complete this food ordering task if it had access to the restaurant API for instance, this would not make it a CUA agent as it is not _interacting_ with the browser to complete the task.

To make this distinction more clear, here is another example task.
Suppose we wanted to find the list of available Airbnbs in Miami from 6/18 to 6/20 for 2 guests.

![airbnb_sc.png](airbnb_sc.png)

How would a browser use agent solve this task:

- **Step 1:** Visit airbnb.com
- **Step 2:** Type "Miami" in the "Where" input box
- **Step 3:** Select "6/18" in the "Check in" date box
- **Step 4:** Select "6/20" in the "Check out" date box
- **Step 5:** Click on the "Who" button
- **Step 6:** Click "+" twice to add two guests
- **Step 7:** Click "Search" button
- **Step 8:** Summarize and extract listings from the webpage

On the other hand, suppose we had an API for Airbnb that looks like: `find_listings(location, check_in, check_out, guests)`

Then a tool-call agent would first need to generate a tool call: `find_listings("Miami", 6/18, 6/20, 2)` and read out the result of the tool call.

Clearly if we had an API for every website and everything on the computer, then it would be much simpler to perform this task. _But that is not the case currently_, many interfaces on the web cannot be accessed by an API and so the only way is through interacting with the website directly. While future interfaces might become more directly accessible to agents via APIs and MCP servers, for now we need to perform direct manipulation with the websites.

## What Does This Tutorial Cover?

In this tutorial, we will cover how to build a basic browser-use agent. The goal of this tutorial is to demystify such agents and show how we can build a simple version of them. The only thing we need is access to a large language model (LLM) that can perform tool calling or structured JSON outputs (GPT-4o, Qwen2.5-VL, Llama 3.1, ...). The LLM does not need to be vision capable, but a model capable of taking image input would improve performance significantly. The LLM also does not need to be trained previously for browser-use, out of the box LLMs can be turned into semi-capable browser-use agents following the recipe in this tutorial. At the end of the tutorial we will discuss further directions.

We will cover three levels of building your browser use agent:

- Level 1: From scratch using only the `playwright` python package.
- Level 2: Using helpers from the `magentic-ui` package which simplifies building your agent.
- Level 3: Using the WebSurfer Agent from the `magentic-ui` package directly.


# Tutorial Prerequisites




You will need Python >3.10 to run this tutorial and the `magentic-ui` package. [Magentic-UI](https://github.com/microsoft/magentic-ui/tree/main) is a research prototype from Microsoft of a human-centered agentic interface. In this tutorial we will be using utilities and helpers from that package without using the Magentic-UI application itself.

We recommend using a virtual environment to avoid conflicts with other packages.

```bash
python3 -m venv .venv
source .venv/bin/activate
pip install magentic-ui
```

Alternatively, if you use [`uv`](https://docs.astral.sh/uv/getting-started/installation/) for dependency management, you can install Magentic-UI with:

```bash
uv venv --python=3.12 .venv
. .venv/bin/activate
uv pip install magentic-ui
```

We also need to install the browsers that our agent will control with playwright:

```bash
playwright install --with-deps chromium
```

The other thing you need to set up is your LLM. The easiest way to follow this tutorial is to obtain an OpenAI API key and set it as an environment variable:

```bash
export OPENAI_API_KEY=<YOUR API KEY>
```

You can also use any open source model with [Ollama](https://ollama.com/) if you have a capable GPU at your disposal. We will be covering both using OpenAI and Ollama.

# Level 1: Building a Browser Use Agent From Scratch

For this level of building our browser use agent, we will only need the `playwright` and `openai` packages which are included in `magentic-ui` package.


## Step 1: Lauching a Browser

The first step is to launch the browser that our agent will control. We will be using the [Playwright](https://github.com/microsoft/playwright-python) library that provides an API to control browsers.

We can launch the browser in headless mode (we cannot see the actual browser on our machine) or non-headless where the browser will be launched locally.


In [ ]:
from playwright.async_api import async_playwright

headless = False  # Change to True to run the browser in headless mode

# Launch and keep browser running
p = await async_playwright().start()
browser = await p.chromium.launch(headless=headless)
# context is the browser window
context = await browser.new_context()
# page is the tab in the browser
page = await context.new_page()
print("Browser launched!")

At this point you should see a browser launched locally, it will be pointing at a blank page:

![blank_page.png](blank_page.png)


We can use the playwright API to interact with this browser, for instance let us navigate to the bing homepage (give it a few seconds)


In [ ]:
await page.goto("https://www.bing.com")
print("Navigated to Bing homepage")

## Step 2: Represent the browser for the Agent using Set-Of-Marks Prompting.

Our next challenge is how do we feed the browser as input to our agent so that it is able to perform actions on it.

Using Playwright we can first take a screenshot of the browser as well as extract the text on the page.


In [ ]:
from IPython.display import Image, display

# Take a screenshot and store it in memory
screenshot_bytes = await page.screenshot()

# Display the screenshot
display(Image(screenshot_bytes))

# Get all the text on the page and print first 10 lines
text = await page.evaluate("() => document.body.innerText")
print("\nFirst 10 lines of text content:")
print("\n".join(text.split("\n")[:10]))

Now how do we get our agent to type in the search box and press search?

The key is to extract all **interactive elements** in the page using Plawright. By interactive elements we mean elements on the page we can interact with including buttons, text boxes, dropdown menus among others. Each interactive element will have an ID that we can track on the page and if it is a visible element it will have the coordinates of bounding box of the element. We will also only look at the interactive elements that are currently visibile in the current viewport, some elements might be out of view and we'd need to scroll down to view them. For simplicity, we will ignore these elements and give our agent the ability to scroll down to view them later on.


In [ ]:
from dataclasses import dataclass
from playwright.async_api import Page


# A class to represent an interactive element on the page
@dataclass
class Element:
    id: int  # The id of the element
    aria_label: (
        str  # The aria-label attribute is used to provide a label for an element
    )
    type: str  # The type of the element
    bbox: dict  # The bounding box of the element
    text: str  # The text content of the element


# We will now go over the page and extract all interactive elements
# We will also add a data attribute to the element with the element ID for later reference
async def get_interactive_elements(page: Page) -> list[Element]:
    elements: list[Element] = []
    # Viewport size is a dict with keys 'width' and 'height'
    viewport_size = page.viewport_size
    print(f"Viewport size: {viewport_size}")

    # For simplicity, we will only look at buttons, textboxes, and links. We can add more roles later on.
    interactive_roles = ["button", "textbox", "link"]
    i = 0
    for role in interactive_roles:
        print(f"Getting {role} elements...")
        # We will use the Playwright API to get all elements with the given role
        elements_with_role = await page.get_by_role(role).all()
        for element in elements_with_role:
            # Check if element is visible and in current viewport
            bbox = await element.bounding_box()
            if bbox:  # Element is visible if it has a bounding box
                # Check if element is in current viewport (not scrolled out of view)
                if 0 <= bbox["y"] <= viewport_size["height"]:
                    # Set a data attribute with the element ID for later reference
                    await element.evaluate(f"el => el.setAttribute('data-element-id', '{i}')")
                    elements.append(
                        Element(
                            id=i,
                            aria_label=await element.get_attribute("aria-label")
                            or await element.get_attribute("aria-role")
                            or "",
                            type=role,
                            bbox=bbox,
                            text=await element.text_content() or "",
                        )
                    )
                    i += 1
    print(f"Found {len(elements)} visible interactive elements in current viewport:")
    return elements


elements = await get_interactive_elements(page)
formatted_list_of_elements = "\n".join(
    [f"Element {i}: {element}" for i, element in enumerate(elements)]
)
print(formatted_list_of_elements)

The first question, is how do we identify the search box element on the Bing page give these elements?
We can try to read to figure this out by reading the list of elements, we can see that it is likely to be Element 19:

Element(id=19, aria_label='0 characters out of 2000', type='textbox', bbox={'x': 193, 'y': 158, 'width': 843, 'height': 22}, text='')

As this is the only texbox or searchbox element on the page.


In [ ]:
# find the search box
search_box_id = None
for element in elements:
    if element.type == "textbox":
        search_box_id = element.id
        break
print(f"Search box id: {search_box_id}")

However, we also have access to the page screenshot and the coordinates of each element. A neat idea would be to superimpose the bounding boxes on top of the screenshot to better understand what each element is. This technique is called Set-of-Mark Prompting (SoM) coined by Yang, Jianwei, et al. [1] to improve visual grounding.

[1]: Yang, Jianwei, et al. "Set-of-mark prompting unleashes extraordinary visual grounding in gpt-4v." arXiv preprint arXiv:2310.11441 (2023). https://arxiv.org/pdf/2310.11441

We're gonna now implement a simplified version of SoM prompting:


In [ ]:
from PIL import Image, ImageDraw
import io


def get_som_screenshot(screenshot_bytes: bytes, elements: list[Element]) -> Image.Image:
    screenshot = Image.open(io.BytesIO(screenshot_bytes))

    # Create a drawing object
    draw = ImageDraw.Draw(screenshot)

    # Draw bounding boxes and element IDs for each element
    for element in elements:
        bbox = element.bbox
        x = bbox["x"]
        y = bbox["y"]
        width = bbox["width"]
        height = bbox["height"]

        # Draw rectangle
        draw.rectangle([(x, y), (x + width, y + height)], outline="red", width=2)

        # Draw element ID
        draw.text((x, y - 15), f"{element.id}", fill="red")

    # Display the annotated screenshot
    display(screenshot)
    som_screenshot = screenshot.copy()
    return som_screenshot


screenshot_bytes = await page.screenshot()
som_screenshot = get_som_screenshot(screenshot_bytes, elements)

This confirms what we previously found that Element with id=19 is in fact the searchbox!

Let us now wrap what we just did in a helper function to prepare the page to be used as input to our agent:


In [99]:
async def prepare_page_for_agent(page: Page) -> tuple[str, str, Image.Image]:
    """
    Prepare the page for the agent.
    Returns:
        tuple[str, str, Image.Image]: The page text, the formatted list of elements, and the screenshot with bounding boxes.
    """
    page_text = await page.evaluate("() => document.body.innerText")
    elements = await get_interactive_elements(page)
    screenshot_bytes = await page.screenshot()
    som_screenshot = get_som_screenshot(screenshot_bytes, elements)

    formatted_list_of_elements = "\n".join(
        [f"Element {i}: {element}" for i, element in enumerate(elements)]
    )

    return page_text, formatted_list_of_elements, som_screenshot


# page_text, formatted_list_of_elements, screenshot = await prepare_page_for_agent(page)

## Step 3: Define Agent Action Space

Now that we have established how to represent the browser state for our Agent, it's time to define our Agent architecture. This section will cover the action space and execution flow that enables our agent to perform tasks using the browser.



### Action Space Definition

Our web agent operates with a carefully designed set of actions that cover the fundamental browser interactions needed for most web automation tasks:

- **`goto(url)`**: Navigate to a specific URL
- **`click(id)`**: Click on an element identified by its ID
- **`type(id, text)`**: Input text into a form field or text element by ID
- **`scroll(direction)`**: Scroll the page vertically (up/down)
- **`stop_action(final_answer)`**: Complete the task and return the final result

_Note: This is a simplfied action set designed for our initial prototype. It can be extended with additional actions like hover, select, wait, etc., as needed._

### Agent Architecture Flow

The following diagram illustrates how our web agent processes user queries and executes actions:

```mermaid
flowchart TD
    A["Input: User Query"] --> B["Initialize Agent"]
    B --> C["Capture Current Page State"]
    C --> D["Analyze Page & Query"]
    D --> E{Action Decision}
    E -->|goto| F["Navigate to URL"]
    E -->|click| G["Click Element by ID"]
    E -->|type| H["Type Text in Element"]
    E -->|scroll| I["Scroll Page"]
    E -->|stop_action| J["Return Final Answer"]
    F --> K["Execute Action"]
    G --> K
    H --> K
    I --> K
    K --> C
    J --> L["Output: Final Answer"]

    style A fill:#e1f5fe
    style L fill:#e8f5e8
    style E fill:#fff3e0
    style J fill:#ffebee
```

### Execution Flow Details

1. **Input Processing**: The agent receives a user query describing the desired task
2. **State Capture**: Current browser page state is captured and processed
3. **Action Selection**: Based on the analysis, one of the five actions is chosen
4. **Execution**: The selected action is executed in the browser. We append the feedback of the action into the chat history.
5. **Loop Continuation**: The process repeats until `stop_action` is triggered

The agent continues this loop until it determines the task is complete, at which point it executes `stop_action` with the final answer.


Our first step is to create the prompt template for the model to decide on the correct action. Instead of using tool calling to decide on the action, we will JSON outputs for simplicity.


In [100]:
AGENT_PROMPT = """
You are a helpful assistant that can navigate a web page and perform actions on it.

The task we are trying to complete is:
{task}

The current visible text on the page is:
{page_text}

The current visible elements on the page are:
{formatted_list_of_elements}

You will need to decide on the next action to take.

The action space is:
- goto(url): navigate to a URL
- click(id): click a button given it's ID
- type(id, text): type "text" into element "id"
- scroll(direction): scroll the page in direction up or down.
- stop_action(final_answer): declare that we have finished the task and prepare a final_answer to return to the user.

Output a JSON object with the following fields:
{{
    "action": "goto" | "click" | "type" | "scroll" | "stop_action",
    "action_args": {{
        "url": "https://www.google.com",
        "id": "123",
        "text": "Hello",
        "direction": "up"
    }}
}}

Only output the JSON object, no other text or comments.
"""

Let's now try this prompt with our LLM:


In [ ]:
from openai import OpenAI
import json
import base64
from PIL import Image
import os

# Prepare the page for the agent
page_text, formatted_list_of_elements, som_screenshot = await prepare_page_for_agent(
    page
)
task = "Search for Magentic-UI"
# Now make the API call
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)  # you can use any other LLM client here
image_data_url = f"data:image/png;base64,{base64.b64encode((lambda b: (som_screenshot.save(b, format='PNG'), b.getvalue())[1])(io.BytesIO())).decode()}"


def get_llm_response(
    client: OpenAI,  # OpenAI client
    task: str,  # Task to complete
    page_text: str,  # Page text
    formatted_list_of_elements: str,  # Formatted list of elements
    image_data_url: str,  # Image data URL
    message_history: list[dict] = [],  # Message history
    model: str = "gpt-4o",  # Model to use
) -> dict:
    response = client.chat.completions.create(
        model=model,
        messages=[
            *message_history,
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": AGENT_PROMPT.format(
                            task=task,
                            page_text=page_text,
                            formatted_list_of_elements=formatted_list_of_elements,
                        ),
                    },
                    {
                        "type": "image_url",
                        "image_url": {"url": image_data_url},
                    },
                ],
            },
        ],
    )

    # Parse the response
    try:
        action_decision = json.loads(response.choices[0].message.content)
        print("Model's decision:", json.dumps(action_decision, indent=2))
    except json.JSONDecodeError:
        # it starts with ```json
        response_content = response.choices[0].message.content
        response_content = response_content.replace("```json", "").replace("```", "")
        action_decision = json.loads(response_content)
        print("Model's decision:", json.dumps(action_decision, indent=2))
    except Exception as e:
        raise e
    return action_decision


action_decision = get_llm_response(
    client, task, page_text, formatted_list_of_elements, image_data_url
)
print(action_decision)

We can see that the model made the right decision given the task of "Search for Magentic-UI", the action is to type in the search box for "Magentic-UI"

The last remaining piece before we put it all together is to now execute the action using Playwright.

## Step 4: Executing the actions with Playwright

For each of the actions we have previously defined, we will now write code using Playwright to execute them.

In [103]:
# This is mostly basic Playwright code, but we will use it to execute the actions.
async def execute_action(action: str, action_args: dict, page: Page) -> str:
    """
    Execute an action on the page.
    """
    if action == "goto":
        await page.goto(action_args["url"])
        return f"I navigated to {action_args['url']}"
    elif action == "click":
        # Get the element using the data attribute
        await page.wait_for_selector(f"[data-element-id='{action_args['id']}']")
        element = page.locator(f"[data-element-id='{action_args['id']}']")
        if element:
            await element.click()
        else:
            raise ValueError(f"Element with ID {action_args['id']} not found")
        return f"I clicked on {action_args['id']}"
    elif action == "type":
        await page.wait_for_selector(f"[data-element-id='{action_args['id']}']")
        element = page.locator(f"[data-element-id='{action_args['id']}']")
        if element:
            await element.fill(action_args["text"])
        else:
            raise ValueError(f"Element with ID {action_args['id']} not found")
        return f"I typed {action_args['text']} into {action_args['id']}"
    elif action == "scroll":
        await page.scroll(action_args["direction"])
        return f"I scrolled {action_args['direction']}"
    elif action == "stop_action":
        return action_args["final_answer"]
    else:
        raise ValueError(f"Invalid action: {action}")

In [ ]:
await execute_action(action_decision["action"], action_decision["action_args"], page)

In [ ]:
# Take a screenshot of the page
screenshot = await page.screenshot()
display(Image.open(io.BytesIO(screenshot)))

Success! We can see that our agent was properly able to type "Magentic-UI" into the searchbox!

The final step is to put it all together into our Agent!

In [ ]:
await browser.close()

## Step 5: Putting it all together into our Agent

In [12]:
from openai import OpenAI
from playwright.async_api import Page
from playwright.async_api import async_playwright
from PIL import Image, ImageDraw
import io
import base64
import json
from dataclasses import dataclass
from IPython.display import display

@dataclass
class Element:
    id: int  # The id of the element
    aria_label: (
        str  # The aria-label attribute is used to provide a label for an element
    )
    type: str  # The type of the element
    bbox: dict  # The bounding box of the element
    text: str  # The text content of the element


AGENT_PROMPT = """
You are a helpful assistant that can navigate a web page and perform actions on it.

The task we are trying to complete is:
{task}

The current visible text on the page is:
{page_text}

The current visible elements on the page are:
{formatted_list_of_elements}

You will need to decide on the next action to take.

The action space is:
- goto(url): navigate to a URL
- click(id): click a button given it's ID
- type(id, text): type "text" into element "id"
- scroll(direction): scroll the page in direction up or down.
- stop_action(final_answer): declare that we have finished the task and prepare a final_answer to return to the user.

Output a JSON object with the following fields:
{{
    "action": "goto" | "click" | "type" | "scroll" | "stop_action",
    "action_args": {{
        "url": "https://www.google.com",
        "id": "123",
        "text": "Hello",
        "direction": "up"
    }}
}}

Only output the JSON object, no other text or comments.
"""


class BrowserUseAgent:
    def __init__(
        self,
        client: OpenAI,
        model: str = "gpt-4o",
        headless: bool = False,
        run_in_jupyter: bool = True,
    ):
        self.client = client
        self.model = model
        self.headless = headless
        self.message_history: list[dict] = []
        self.page: Page = None
        self.run_in_jupyter = run_in_jupyter

    async def _launch_browser(self) -> None:
        p = await async_playwright().start()
        self.browser = await p.chromium.launch(headless=self.headless)
        # context is the browser window
        self.context = await self.browser.new_context()
        # page is the tab in the browser
        self.page = await self.context.new_page()

    async def execute_task(self, task: str) -> str:
        """
        This is NEW! This is the main function that will be called to execute the task and implement our agent loop.
        """
        # Step 1: Launch the browser if it's not already launched
        if self.page is None:
            await self._launch_browser()
        # Our stop condition is when the LLM decides to output stop_action
        should_stop = False
        final_answer = None
        i = 0
        while not should_stop:
            # Step 2: Prepare the page for the agent
            (
                page_text,
                formatted_list_of_elements,
                som_screenshot,
            ) = await self._prepare_page_for_agent(self.page)
            # Step 3: Get the LLM response
            image_data_url = f"data:image/png;base64,{base64.b64encode((lambda b: (som_screenshot.save(b, format='PNG'), b.getvalue())[1])(io.BytesIO())).decode()}"
            action_decision = self._get_llm_response(
                self.client,
                task,
                page_text,
                formatted_list_of_elements,
                image_data_url,
                self.message_history,
                self.model,
            )
            print(f"Action decision {i}: {action_decision}")
            # Add the action decision to the message history
            self.message_history.append(
                {
                    "role": "user",
                    "content": [{"type": "text", "text": json.dumps(action_decision)}],
                }
            )
            # Step 4: Execute the action with some error handling
            try:
                action_feedback = await self._execute_action(
                    action_decision["action"], action_decision["action_args"], self.page
                )
            except Exception as e:
                print(f"Error executing action {i}: {e}")
                action_feedback = f"Error executing action {i}: {e}"
            print(f"Action feedback {i}: {action_feedback}")
            # Sleep for 3 seconds to let the page load
            await self.page.wait_for_timeout(3000)
            # Update the message history with feedback on the action and the new page screenshot
            new_page_screenshot = await self.page.screenshot()
            self.message_history.append(
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": action_feedback},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{base64.b64encode(new_page_screenshot).decode()}"
                            },
                        },
                    ],
                }
            )
            if self.run_in_jupyter:
                display(Image.open(io.BytesIO(new_page_screenshot)))
            # Check if the task is complete
            should_stop = action_decision["action"] == "stop_action"
            if should_stop:
                final_answer = action_decision["action_args"]["final_answer"]
            i += 1
        return final_answer

    async def _execute_action(self, action: str, action_args: dict, page: Page) -> str:
        """
        Execute an action on the page.
        """
        if action == "goto":
            await page.goto(action_args["url"])
            return f"I navigated to {action_args['url']}"
        elif action == "click":
            # Get the element using the data attribute
            await page.wait_for_selector(f"[data-element-id='{action_args['id']}']")
            element = page.locator(f"[data-element-id='{action_args['id']}']")
            if element:
                await element.click()
            else:
                raise ValueError(f"Element with ID {action_args['id']} not found")
            return f"I clicked on {action_args['id']}"
        elif action == "type":
            await page.wait_for_selector(f"[data-element-id='{action_args['id']}']")
            element = page.locator(f"[data-element-id='{action_args['id']}']")
            if element:
                await element.fill(action_args["text"])
                # Press enter
                await element.press("Enter")
            else:
                raise ValueError(f"Element with ID {action_args['id']} not found")
            return f"I typed {action_args['text']} into {action_args['id']}"
        elif action == "scroll":
            await page.scroll(action_args["direction"])
            return f"I scrolled {action_args['direction']}"
        elif action == "stop_action":
            return action_args["final_answer"]
        else:
            raise ValueError(f"Invalid action: {action}")

    def _get_llm_response(
        self,
        client: OpenAI,  # OpenAI client
        task: str,  # Task to complete
        page_text: str,  # Page text
        formatted_list_of_elements: str,  # Formatted list of elements
        image_data_url: str,  # Image data URL
        message_history: list[dict] = [],  # Message history
        model: str = "gpt-4o",  # Model to use
    ) -> dict:
        response = client.chat.completions.create(
            model=model,
            messages=[
                *message_history,
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": AGENT_PROMPT.format(
                                task=task,
                                page_text=page_text,
                                formatted_list_of_elements=formatted_list_of_elements,
                            ),
                        },
                        {
                            "type": "image_url",
                            "image_url": {"url": image_data_url},
                        },
                    ],
                },
            ],
        )

        # Parse the response
        try:
            action_decision = json.loads(response.choices[0].message.content)
        except json.JSONDecodeError:
            # it starts with ```json
            response_content = response.choices[0].message.content
            response_content = response_content.replace("```json", "").replace(
                "```", ""
            )
            action_decision = json.loads(response_content)
        except Exception as e:
            raise e
        return action_decision

    async def _prepare_page_for_agent(self, page: Page) -> tuple[str, str, Image.Image]:
        """
        Prepare the page for the agent.
        Returns:
            tuple[str, str, Image.Image]: The page text, the formatted list of elements, and the screenshot with bounding boxes.
        """
        page_text = await page.evaluate("() => document.body.innerText")
        elements = await self._get_interactive_elements(page)
        screenshot_bytes = await page.screenshot()
        som_screenshot = self._get_som_screenshot(screenshot_bytes, elements)

        formatted_list_of_elements = "\n".join(
            [f"Element {i}: {element}" for i, element in enumerate(elements)]
        )

        return page_text, formatted_list_of_elements, som_screenshot

    def _get_som_screenshot(
        self, screenshot_bytes: bytes, elements: list[Element]
    ) -> Image.Image:
        screenshot = Image.open(io.BytesIO(screenshot_bytes))

        # Create a drawing object
        draw = ImageDraw.Draw(screenshot)

        # Draw bounding boxes and element IDs for each element
        for element in elements:
            bbox = element.bbox
            x = bbox["x"]
            y = bbox["y"]
            width = bbox["width"]
            height = bbox["height"]

            # Draw rectangle
            draw.rectangle([(x, y), (x + width, y + height)], outline="red", width=2)

            # Draw element ID
            draw.text((x, y - 15), f"{element.id}", fill="red")

        som_screenshot = screenshot.copy()
        return som_screenshot

    async def _get_interactive_elements(self, page: Page) -> list[Element]:
        elements: list[Element] = []
        # Viewport size is a dict with keys 'width' and 'height'
        viewport_size = page.viewport_size

        # For simplicity, we will only look at buttons, textboxes, and links. We can add more roles later on.
        interactive_roles = ["button", "textbox", "link"]
        i = 0
        for role in interactive_roles:
            # We will use the Playwright API to get all elements with the given role
            elements_with_role = await page.get_by_role(role).all()
            for element in elements_with_role:
                # Check if element is visible and in current viewport
                bbox = await element.bounding_box()
                if bbox:  # Element is visible if it has a bounding box
                    # Check if element is in current viewport (not scrolled out of view)
                    if 0 <= bbox["y"] <= viewport_size["height"]:
                        # Set a data attribute with the element ID for later reference
                        await element.evaluate(
                            f"el => el.setAttribute('data-element-id', '{i}')"
                        )
                        elements.append(
                            Element(
                                id=i,
                                aria_label=await element.get_attribute("aria-label")
                                or await element.get_attribute("aria-role")
                                or "",
                                type=role,
                                bbox=bbox,
                                text=await element.text_content() or "",
                            )
                        )
                        i += 1
        return elements

    async def close(self) -> None:
        if self.page is not None:
            await self.page.close()
        if self.context is not None:
            await self.context.close()
        if self.browser is not None:
            await self.browser.close()

Now let's run the Agent on a sample task!

In [ ]:
from openai import OpenAI
import os
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
agent = BrowserUseAgent(openai_client)
try:
    final_answer = await agent.execute_task("find the open issues assigned to husseinmozannar on the microsoft/magentic-ui repo on github")
    print(final_answer)
finally:
    await agent.close()

Sucess! Our agent was able to navigate to GitHub and filter the issues assigned to me. It ran into some issues but was able to debug and get to the right answer.

To conclude, in this short tutorial, we showed how to build a browser use agent from scratch.

The main ingredients were: set-of-marks prompting, playwright for browser automation and tool calling or structured JSON output ability of current LLMs. With these three ingredients we can build a semi-capable agent to navigate the web!

# Level 2: Building a Browser Use Agent Using Magentic-UI

While it was fun building the browser use agent from scratch, it was not easy. We had to figure out how to launch the browser, fiddle around with playwright to extract interactive elements, figure out how to execute actions on the page and so on.

The `magentic-ui` library as we will see has many utilities that will make your life much easier when building browser use agents. We will now do the same steps as before but by using the helpers from the `magentic-ui` library.


## Step 1: Launching a Browser

Magentic-UI provides three different Playwright browser implementations, each designed for specific use cases:

1. Local Playwright Browser (`LocalPlaywrightBrowser`)
- **Purpose**: Runs Playwright directly on the local machine without Docker
- **Use Case**: Development and testing environments where Docker isn't needed
- **Features**: Lightweight, direct browser control, supports both headless and headed modes

2. Headless Docker Playwright Browser (`HeadlessDockerPlaywrightBrowser`)  
- **Purpose**: Runs a headless Playwright browser inside a Docker container
- **Use Case**: Production environments, CI/CD pipelines, server-side automation
- **Features**: Isolated execution, reproducible environment, no GUI overhead and more secure.
- **Docker Image**: Uses Microsoft's official Playwright Docker image (`mcr.microsoft.com/playwright:v1.51.1-noble`)

3. VNC Docker Playwright Browser (`VncDockerPlaywrightBrowser`)
- **Purpose**: Runs Playwright in Docker with VNC support for visual interaction, you can interact with the browser on localhost.
- **Use Case**: Debugging, development, and scenarios requiring visual browser inspection
- **Features**: Programmatic control + visual access via noVNC web interface
- **Docker Image**: Uses custom `magentic-ui-vnc-browser` image with VNC server. You need to run `magentic-ui --rebuild-docker` command to build it.

How to Launch Each Browser:

```python
from pathlib import Path
from magentic_ui.tools.playwright import HeadlessDockerPlaywrightBrowser, VncDockerPlaywrightBrowser, LocalPlaywrightBrowser

# Direct instantiation examples
async def launch_browsers():
    # Headless Docker Browser
    headless_browser = HeadlessDockerPlaywrightBrowser(
        playwright_port=37367,
        inside_docker=False
    )
    
    # VNC Docker Browser  
    vnc_browser = VncDockerPlaywrightBrowser(
        bind_dir=Path("./workspace"),
        playwright_port=37367,
        novnc_port=6080,
        inside_docker=False
    )
    
    # Local Browser
    local_browser = LocalPlaywrightBrowser(headless=True)
    
```



For simplicity we will stick with the local playwright browser that we launched in Level 1:

In [ ]:
from magentic_ui.tools.playwright import LocalPlaywrightBrowser
browser = LocalPlaywrightBrowser(headless=False)
# Start the browser
await browser._start()
# Get the browser context and start a new page
context = browser.browser_context
page = await context.new_page()


You should now see a browser open to the blank page.

## Step 2: Represent the browser for the Agent using Set-Of-Marks Prompting.

To get the interactive elements on the page, we have done a lot of work for you in Magentic-UI to capture every posible interactive element type on the page including elements in the shadow-DOM [(see this javascript file if interested for more info)](https://github.com/microsoft/magentic-ui/blob/main/src/magentic_ui/tools/playwright/page_script.js).

These utilities are wrapped in a helper class called the [`PlaywrightController`](https://github.com/microsoft/magentic-ui/blob/main/src/magentic_ui/tools/playwright/playwright_controller.py)

In [31]:
from magentic_ui.tools.playwright import PlaywrightController
browser_controller = PlaywrightController(viewport_width=1280, viewport_height=720)

The PlaywrightController has a lot of convenience methods that have been debugged extensively so that we can perform actions on the browser more reliably and securily.

There are methods to get the interactive elements, get the screenshot, click, type, scroll, manage tabs, hover, describe pages in markdown and much more.

For now, let's navigate to Bing using our `browser_controller`.

In [33]:
_ = await browser_controller.visit_page(page, "https://www.bing.com")

The visit_page method only returns when the page is fully loaded.

Now let us get the set of interactive elements:

In [ ]:
interactive_elements = await browser_controller.get_interactive_rects(page)
# print the first 20 interactive elements
i = 0
for element in interactive_elements:
    print(f"Element {i}: id={element}, data={interactive_elements[element]}")
    i += 1
    if i > 20:
        break

You'll notice that this ran much faster than using the Playwright script in Level 1 tutorial because here we are using javascript to extract the elements instead of going through the playwright API.

Our searchbox is now Element id 22 and has the following data:


    Element 12: id=22, data={'tag_name': 'textarea', 'role': 'textbox', 'aria_name': '0 characters out of 2000', 'v_scrollable': False, 'rects': [{'x': 193, 'y': 158, 'width': 843, 'height': 22, 'top': 158, 'right': 1036, 'bottom': 180, 'left': 193}]}

To type in the searchbox we can use the fill_id method of the PlaywrightController:

In [38]:

await browser_controller.fill_id(page, "22", "Magentic-UI")


Let's check if we are the right page:

In [ ]:
from PIL import Image
import io
from IPython.display import display

screenshot = await browser_controller.get_screenshot(page)
image = Image.open(io.BytesIO(screenshot))
display(image)


We can also easily extract the search results using the get_page_markdown method that uses the [`markitdown`](https://github.com/microsoft/markitdown) package from our team at Microsoft Research.

In [ ]:
page_text = await browser_controller.get_page_markdown(page)
print(page_text)

The final thing we need is to get the set-of-marks image:

In [ ]:
from magentic_ui.agents.web_surfer._set_of_mark import add_set_of_mark


interactive_elements = await browser_controller.get_interactive_rects(page)
screenshot = await browser_controller.get_screenshot(page)
som_screenshot, visible_elements, elements_above, elements_below, _ = add_set_of_mark(
    screenshot, interactive_elements, use_sequential_ids=True
)

display(som_screenshot)


The add_set_of_mark method returns the SoM screenshot in addition to elements visible on the viewport, elements above the viewport and elements below the viewport.

We can see how much the `magentic-ui` makes our life easier with these tools, we are now ready to re-implement the agent from Level 1!

## Step 3: Putting it all together

Using the tools from the `magentic-ui` library now we can more easily implement our BrowserUseAgent:

In [71]:
from openai import OpenAI
from playwright.async_api import Page
from playwright.async_api import async_playwright
from PIL import Image, ImageDraw
import io
import base64
import json
from dataclasses import dataclass
from IPython.display import display
from magentic_ui.tools.playwright import LocalPlaywrightBrowser
from magentic_ui.tools.playwright import PlaywrightController
from magentic_ui.agents.web_surfer._set_of_mark import add_set_of_mark


AGENT_PROMPT = """
You are a helpful assistant that can navigate a web page and perform actions on it.

The task we are trying to complete is:
{task}

The current visible text on the page is:
{page_text}

The current visible elements on the page are:
{formatted_list_of_elements}

You will need to decide on the next action to take.

The action space is:
- goto(url): navigate to a URL
- click(id): click a button given it's ID
- type(id, text): type "text" into element "id"
- scroll(direction): scroll the page in direction up or down.
- stop_action(final_answer): declare that we have finished the task and prepare a final_answer to return to the user.

Output a JSON object with the following fields:
{{
    "action": "goto" | "click" | "type" | "scroll" | "stop_action",
    "action_args": {{
        "url": "https://www.google.com",
        "id": "123",
        "text": "Hello",
        "direction": "up"
    }}
}}

Only output the JSON object, no other text or comments.
"""


class BrowserUseAgent:
    def __init__(
        self,
        client: OpenAI,
        model: str = "gpt-4o",
        headless: bool = False,
        run_in_jupyter: bool = True,
    ):
        self.client = client
        self.model = model
        self.headless = headless
        self.message_history: list[dict] = []
        self.page: Page = None
        self.run_in_jupyter = run_in_jupyter
        self.browser_controller = PlaywrightController(
            viewport_width=1280, viewport_height=720
        )

    async def _launch_browser(self) -> None:
        self.browser = LocalPlaywrightBrowser(headless=False)
        # Start the browser
        await self.browser._start()
        # Get the browser context and start a new page
        self.context = self.browser.browser_context
        self.page = await self.context.new_page()

    async def execute_task(self, task: str) -> str:
        """
        This is NEW! This is the main function that will be called to execute the task and implement our agent loop.
        """
        # Step 1: Launch the browser if it's not already launched
        if self.page is None:
            await self._launch_browser()
        # Our stop condition is when the LLM decides to output stop_action
        should_stop = False
        final_answer = None
        i = 0
        while not should_stop:
            # Step 2: Prepare the page for the agent
            (
                page_text,
                formatted_list_of_elements,
                som_screenshot,
            ) = await self._prepare_page_for_agent(self.page)
            # Step 3: Get the LLM response
            image_data_url = f"data:image/png;base64,{base64.b64encode((lambda b: (som_screenshot.save(b, format='PNG'), b.getvalue())[1])(io.BytesIO())).decode()}"
            action_decision = self._get_llm_response(
                self.client,
                task,
                page_text,
                formatted_list_of_elements,
                image_data_url,
                self.message_history,
                self.model,
            )
            print(f"Action decision {i}: {action_decision}")
            # Add the action decision to the message history
            self.message_history.append(
                {
                    "role": "user",
                    "content": [{"type": "text", "text": json.dumps(action_decision)}],
                }
            )
            # Step 4: Execute the action with some error handling
            try:
                action_feedback = await self._execute_action(
                    action_decision["action"], action_decision["action_args"], self.page
                )
            except Exception as e:
                print(f"Error executing action {i}: {e}")
                action_feedback = f"Error executing action {i}: {e}"
            print(f"Action feedback {i}: {action_feedback}")
            # Sleep for 3 seconds to let the page load
            await self.page.wait_for_timeout(3000)
            # Update the message history with feedback on the action and the new page screenshot
            new_page_screenshot = await self.page.screenshot()
            self.message_history.append(
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": action_feedback},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{base64.b64encode(new_page_screenshot).decode()}"
                            },
                        },
                    ],
                }
            )
            if self.run_in_jupyter:
                display(Image.open(io.BytesIO(new_page_screenshot)))
            # Check if the task is complete
            should_stop = action_decision["action"] == "stop_action"
            if should_stop:
                final_answer = action_decision["action_args"]["final_answer"]
            i += 1
        return final_answer

    async def _prepare_page_for_agent(self, page: Page) -> tuple[str, str, bytes]:
        interactive_elements = await self.browser_controller.get_interactive_rects(page)
        screenshot = await self.browser_controller.get_screenshot(page)
        som_screenshot, visible_elements, elements_above, elements_below, _ = (
            add_set_of_mark(screenshot, interactive_elements, use_sequential_ids=False)
        )
        visible_elements_formatted = ""
        for element_id in visible_elements:
            element_data = interactive_elements[element_id]
            visible_elements_formatted += f"{element_id}: {element_data}\n"

        page_text = await self.browser_controller.get_page_markdown(page)
        return page_text, visible_elements_formatted, som_screenshot
    async def _execute_action(self, action: str, action_args: dict, page: Page) -> str:
        if action == "goto":
            await self.browser_controller.visit_page(page, action_args["url"])
            return f"Visited {action_args['url']}"
        elif action == "click":
            await self.browser_controller.click_id(self.context, page, action_args["id"])
            return f"Clicked {action_args['id']}"
        elif action == "type":
            await self.browser_controller.fill_id(page, action_args["id"], action_args["text"])
            return f"Typed {action_args['text']} into {action_args['id']}"
        elif action == "scroll":
            if action_args["direction"] == "up":
                await self.browser_controller.page_up(page)
            elif action_args["direction"] == "down":
                await self.browser_controller.page_down(page)
            return f"Scrolled {action_args['direction']}"
        elif action == "stop_action":
            return action_args["final_answer"]
        else:
            raise ValueError(f"Invalid action: {action}")

    def _get_llm_response(
        self,
        client: OpenAI,  # OpenAI client
        task: str,  # Task to complete
        page_text: str,  # Page text
        formatted_list_of_elements: str,  # Formatted list of elements
        image_data_url: str,  # Image data URL
        message_history: list[dict] = [],  # Message history
        model: str = "gpt-4o",  # Model to use
    ) -> dict:
        response = client.chat.completions.create(
            model=model,
            messages=[
                *message_history,
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": AGENT_PROMPT.format(
                                task=task,
                                page_text=page_text,
                                formatted_list_of_elements=formatted_list_of_elements,
                            ),
                        },
                        {
                            "type": "image_url",
                            "image_url": {"url": image_data_url},
                        },
                    ],
                },
            ],
        )

        # Parse the response
        try:
            action_decision = json.loads(response.choices[0].message.content)
        except json.JSONDecodeError:
            # it starts with ```json
            response_content = response.choices[0].message.content
            response_content = response_content.replace("```json", "").replace(
                "```", ""
            )
            action_decision = json.loads(response_content)
        except Exception as e:
            raise e
        return action_decision

    async def close(self) -> None:
        if self.page is not None:
            await self.page.close()
        if self.context is not None:
            await self.context.close()


In [ ]:
from openai import OpenAI
import os
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
agent = BrowserUseAgent(openai_client)
try:
    final_answer = await agent.execute_task("find the open issues assigned to husseinmozannar on the microsoft/magentic-ui repo on github")
    print(final_answer)
finally:
    await agent.close()

Success! Our agent again performed the task correctly!

With this tutorial, I hope to have convinced you that `magentic-ui` can help you build a browser-use agent more easily. You might be curious how to build the best browser-use agent possible given this, and we have already implemented one for you with many features that we haven't discussed previously in Magentic-UI which we will discuss next.

# Level 3: Using the WebSurfer Agent from Magentic-UI

We have a reference implementation of a capable browser use agent in Magentic-UI which we call the `WebSurfer` agent. I'll show you now how to use it. 



`WebSurfer` is an AutoGen AgentChat agent built using the tools we have seen previously to complete actions autonomously on the web. We have spent a lot of time fixing many many edge cases that arise on the web to arrive at a more reliable (but not perfect) browser use agent.
This agent builds on the [`MultimodalWebSurfer`](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.agents.web_surfer.html) agent from AutoGen that we previously developed. 

Let's see now how to use it!

In [ ]:

from autogen_ext.models.openai import OpenAIChatCompletionClient
from magentic_ui.agents import WebSurfer
from magentic_ui.tools.playwright import (
    LocalPlaywrightBrowser,
)

browser = LocalPlaywrightBrowser(headless=False)

model_client = OpenAIChatCompletionClient(model="gpt-4o")

web_surfer = WebSurfer(
    name="web_surfer",
    model_client=model_client, # Use any client from AutoGen!
    animate_actions=True, # Set to True if you want to see the actions being animated!
    max_actions_per_step=10, # Maximum number of actions to perform before returning
    downloads_folder="debug", # Where to save downloads
    debug_dir="debug", # Where to save debug files and screenshots
    to_save_screenshots=False, # set to True if you want to save screenshots of the actions
    browser=browser, # Use any browser from Magentic-UI!
    multiple_tools_per_call=False, # Set to True if you want to use multiple tools per call
    json_model_output=False, # Set to True if your model does not support tool calling
)
await web_surfer.lazy_init()

task = "find the open issues assigned to husseinmozannar on the microsoft/magentic-ui repo on github"
try:
    messages = []
    async for message in web_surfer.run_stream(task=task):
        messages.append(message)
        print(message)
    print("########################################################")
    print("Final answer:")
    print(messages[-1].messages[-2].content)
finally:
    await web_surfer.close()


We encourage you to experiment using the sample code file [sample_web_surfer.py](https://github.com/microsoft/magentic-ui/blob/main/samples/sample_web_surfer.py) and to use the Magentic-UI application which provides a web UI to interact with the WebSurfer agent and launch multiple parallel tasks and more!

Just run:

```bash
python3 -m venv .venv
source .venv/bin/activate
pip install magentic-ui
# export OPENAI_API_KEY=<YOUR API KEY>
magentic ui --port 8081
```
See [https://github.com/microsoft/magentic-ui](https://github.com/microsoft/magentic-ui) for the full instructions.

![../img/magenticui_running.png](../img/magenticui_running.png)

# What's next?





## Evaluation

The first thing you might be curious about is how well does the WebSurfer agent perform?

In Magentic-UI, we have built a small evaluation library [magentic-ui/eval](https://github.com/microsoft/magentic-ui/tree/main/src/magentic_ui/eval) that implements popular browser-use benchmarks and makes it easy to run evals. We will be building a bit on this library and will have a tutorial on how to use it.

 Magentic-UI has been tested against several benchmarks when running with o4-mini: [GAIA](https://huggingface.co/datasets/gaia-benchmark/GAIA) test set (42.52%), which assesses general AI assistants across reasoning, tool use, and web interaction tasks ; [AssistantBench](https://huggingface.co/AssistantBench) test set (27.60%), focusing on realistic, time-consuming web tasks; [WebVoyager](https://github.com/MinorJerry/WebVoyager) (82.2%), measuring end-to-end web navigation in real-world scenarios; and [WebGames](https://webgames.convergence.ai/) (45.5%), evaluating general-purpose web-browsing agents through interactive challenges.
To reproduce these experimental results, please see the following [instructions](experiments/README.md).

For reference, the current SOTA on WebVoyager is the [browser-use library](https://browser-use.com/posts/sota-technical-report) using GPT-4o achieving 89%. Note that the WebVoyager evaluation is not consistent across different systems as it relies on a mix of LLM-as-a-judge evaluation and human evaluation.


## Limitations

Using the Set-Of-Mark approach for building the Browser Use Agent has many limitations (note that both Magentic-UI and Browser Use library use SoM). For instance, any task that requires understanding coordinates on the screen our agent will fail on.
Examples:

- dragging an element from position A to position B
- drawing on the screen
- playing web games

Moreover, it will not generalize to any Computer Use task where we might not have the DOM to obtain element coordinates. Therefore, we will need to have a model that can click on specific coordinates rather than using element IDs. The [UI-Tars](https://github.com/bytedance/UI-TARS) models have such an ability as well as the latest [compute-preview-api](https://platform.openai.com/docs/guides/tools-computer-use) from OpenAI. Another approach is to use a grounding or parsing model instead of the DOM such as [OmniParser](https://microsoft.github.io/OmniParser/) to obtain element IDs from any GUI interface combined with a tool-calling LLM.


Another limitation is that these agents are not *real-time* and so tasks such as video-understanding or playing games become almost impossible natively as there multiple seconds delay between each agent action.

## Safety

Current LLMs are still very prone to adversarial attacks on the web, see these papers for how bad things can get it with current models even those tuned directly for CUA:

- [Commercial LLM Agents Are Already Vulnerable to Simple Yet Dangerous Attacks
](https://arxiv.org/html/2502.08586v1)
- [RedTeamCUA:
Realistic Adversarial Testing of Computer-Use Agents in
Hybrid Web-OS Environments](https://osu-nlp-group.github.io/RedTeamCUA/)

We recommend to have guardrails built into the agent to allow the human to approve actions if needed. We call such guardrails "ActionGuard" in Magentic-UI and they allow you to define heuristics in addition to LLM judgmenet for when actions might need human approval.




If you've made it this far I really appreciate you taking the time to read and hope you've enjoyed following along!